# Import


In [5]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import lightning as L
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger

import os
import numpy as np
import random

# Dataset


In [6]:
root_path = "/Users/ms/cs/ML/NeuroImagen/"
dataset_path = os.path.join(root_path, "dataset")
images_dataset_path = os.path.join(dataset_path, "imageNet_images")
eeg_dataset_path = os.path.join(dataset_path, "eeg")

In [7]:
class EEGDataset(Dataset):
    def __init__(self, eeg_dataset_file_name="eeg_5_95_std.pth") -> None:
        super().__init__()
        loaded = torch.load(os.path.join(eeg_dataset_path, eeg_dataset_file_name))
        self.data = loaded["dataset"]
        self.labels = loaded["labels"]
        self.images = loaded["images"]
        self.size = len(self.data)

    def __getitem__(self, idx):
        # t() -> transpose
        eeg = self.data[idx]["eeg"].t()
        eeg = eeg[20:460, :]

        label = self.data[idx]["label"]
        return eeg, label

    def __len__(self):
        return self.size

In [8]:
class Splitter(Dataset):
    def __init__(self, dataset, split_name="train") -> None:
        super().__init__()
        self.dataset = dataset

        loaded = torch.load(
            os.path.join(eeg_dataset_path, "block_splits_by_image_all.pth")
        )
        self.target_data_indices = loaded["splits"][0][split_name]
        # filter data that is too short
        self.target_data_indices = [
            i
            for i in self.target_data_indices
            if 450 <= self.dataset.data[i]["eeg"].size(1) <= 600
        ]

        self.size = len(self.target_data_indices)
        self.all_labels = np.array(self.get_all_labels())
        self.all_eegs = self.get_all_eegs()

    def __getitem__(self, idx):
        eeg, label = self.dataset[self.target_data_indices[idx]]
        return eeg, label

    def __len__(self):
        return self.size

    def get_all_labels(self):
        data = [self.dataset[idx] for idx in self.target_data_indices]
        return [item[1] for item in data]

    def get_all_eegs(self):
        data = [self.dataset[idx] for idx in self.target_data_indices]
        return [item[0] for item in data]

    def generate_data_points(self, anchor_labels, positive=True):
        eeg_shape = self.__getitem__(0)[0].size()
        eegs = torch.empty(0, eeg_shape[0], eeg_shape[1])
        labels = torch.empty(0)
        for anchor_label in anchor_labels:
            indices = (
                np.argwhere(self.all_labels == anchor_label.item())[:, 0]
                if positive
                else np.argwhere(self.all_labels != anchor_label.item())[:, 0]
            )
            data_idx = np.random.choice(indices)
            eeg = self.all_eegs[data_idx]
            eegs = torch.cat((eegs, eeg.unsqueeze(dim=0)))
            labels = torch.cat(
                (labels, torch.tensor(self.all_labels[data_idx]).unsqueeze(dim=0))
            )

        return eegs, labels

    def get_data(self, anchor_label, positive: bool = True):
        cnt = 0
        while True:
            idx = random.choice(self.target_data_indices)
            if positive and self.dataset[idx][1] == anchor_label:
                return self.dataset[idx]
            if not positive and self.dataset[idx][1] != anchor_label:
                return self.dataset[idx]

            if cnt >= 2000:
                raise Exception(f"get_data failed after {cnt} tries")
            cnt += 1

In [12]:
dataset = EEGDataset(eeg_dataset_file_name="eeg_5_95_std.pth")
loaders = {
    split: DataLoader(
        Splitter(dataset, split_name=split), batch_size=16, shuffle=True, drop_last=True
    )
    for split in ["train", "val", "test"]
}

# Model


In [13]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

config = {}

mps


In [14]:
class FeatureExtractor_ContrastiveLearning_NN(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()

        # Triplet loss
        def dist_fn(x1, x2):
            return torch.sum(torch.pow(torch.subtract(x1, x2), 2), dim=0)

        self.loss_fn = nn.TripletMarginWithDistanceLoss(
            distance_function=dist_fn, margin=config["margin"]
        )

        # model
        self.input_size = 128
        self.hidden_size = 128
        self.lstm_layers = 1
        self.out_size = 128

        self.lstm = nn.LSTM(
            self.input_size,
            self.hidden_size,
            num_layers=self.lstm_layers,
            batch_first=True,
        )
        self.output = nn.Sequential(
            nn.Linear(in_features=self.hidden_size, out_features=self.out_size),
            nn.ReLU(),
        )

    def forward(self, input):
        input = input.to(device)

        lstm_out, _ = self.lstm(input)
        res = self.output(lstm_out[:, -1, :])
        return res

    def training_step(self, batch, batch_idx):
        anchor_eeg, anchor_label = batch

        # positive_eeg = torch.empty(
        #     (0, anchor_eeg[0].shape[0], anchor_eeg[0].shape[1]), dtype=torch.float32
        # )
        # negative_eeg = torch.empty(
        #     (0, anchor_eeg[0].shape[0], anchor_eeg[0].shape[1]), dtype=torch.float32
        # )
        # for label in anchor_label:
        #     positive_eeg = torch.cat(
        #         (
        #             positive_eeg,
        #             loaders["train"]
        #             .dataset.get_data(label, positive=True)[0]
        #             .unsqueeze(dim=0),
        #         )
        #     )
        #     negative_eeg = torch.cat(
        #         (
        #             negative_eeg,
        #             loaders["train"]
        #             .dataset.get_data(label, positive=False)[0]
        #             .unsqueeze(dim=0),
        #         )
        #     )

        positive_eeg, positive_label = loaders["train"].dataset.generate_data_points(
            anchor_label, positive=True
        )
        negative_eeg, negative_label = loaders["train"].dataset.generate_data_points(
            anchor_label, positive=False
        )

        anchor_feature = self(anchor_eeg)
        positive_feature = self(positive_eeg)
        negative_eeg = self(negative_eeg)

        loss = self.loss_fn(anchor_feature, positive_feature, negative_eeg)

        self.log_dict(
            {"train_loss": loss, "lr": self.scheduler.get_last_lr()[0]},
            on_epoch=True,
            prog_bar=True,
        )
        return loss

    def validation_step(self, batch, batch_idx):
        anchor_eeg, anchor_label = batch

        # positive_eeg = torch.empty(
        #     (0, anchor_eeg[0].shape[0], anchor_eeg[0].shape[1]), dtype=torch.float32
        # )
        # negative_eeg = torch.empty(
        #     (0, anchor_eeg[0].shape[0], anchor_eeg[0].shape[1]), dtype=torch.float32
        # )
        # for label in anchor_label:
        #     positive_eeg = torch.cat(
        #         (
        #             positive_eeg,
        #             loaders["val"]
        #             .dataset.get_data(label, positive=True)[0]
        #             .unsqueeze(dim=0),
        #         )
        #     )
        #     negative_eeg = torch.cat(
        #         (
        #             negative_eeg,
        #             loaders["val"]
        #             .dataset.get_data(label, positive=False)[0]
        #             .unsqueeze(dim=0),
        #         )
        #     )

        positive_eeg, positive_label = loaders["val"].dataset.generate_data_points(
            anchor_label, positive=True
        )
        negative_eeg, negative_label = loaders["val"].dataset.generate_data_points(
            anchor_label, positive=False
        )

        anchor_feature = self(anchor_eeg)
        positive_feature = self(positive_eeg)
        negative_eeg = self(negative_eeg)

        loss = self.loss_fn(anchor_feature, positive_feature, negative_eeg)

        self.log_dict(
            {"val_loss": loss},
            on_epoch=True,
            prog_bar=True,
        )

    def create_optimizer(self):
        if config["optimizer"] == "Adam":
            return optim.Adam(
                self.parameters(), lr=config["lr"], weight_decay=config["weight-decay"]
            )
        elif config["optimizer"] == "AdamW":
            return optim.AdamW(
                self.parameters(), lr=config["lr"], weight_decay=config["weight-decay"]
            )
        elif config["optimizer"] == "SGD":
            return optim.SGD(
                self.parameters(), lr=config["lr"], weight_decay=config["weight-decay"]
            )
        else:
            raise Exception("optimizer config error")

    def create_scheduler(self, optimizer):
        if config["scheduler"] == "LambdaLR":
            return optim.lr_scheduler.LambdaLR(
                optimizer, lambda epoch: config["lambda-factor"] ** epoch
            )
        else:
            raise Exception("scheduler config error")

    def configure_optimizers(self):
        # optimizer = optim.Adam(self.parameters(), lr=1e-4)
        # scheduler = optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.95**epoch)
        optimizer = self.create_optimizer()
        scheduler = self.create_scheduler(optimizer)
        self.scheduler = scheduler
        return [optimizer], [scheduler]

# Training


In [15]:
config = {
    "optimizer": "Adam",  # ("Adam", "AdamW", "SGD")
    "lr": 1e-4,
    "scheduler": "LambdaLR",
    "lambda-factor": 0.95,
    "weight-decay": 0,
    "margin": 1.5,
}

In [16]:
model = FeatureExtractor_ContrastiveLearning_NN()
model.to(device)

logger = TensorBoardLogger(
    save_dir="/Users/ms/cs/ML/NeuroImagen/lightning_logs/ContrastiveLossFeatureLearning",
    name=f"{config['optimizer']}_{config['lr']}_{config['scheduler']}_margin_{config['margin']}",
    version=f"weight-decay_{config['weight-decay']}_lambda-factor_{config['lambda-factor']}",
)

lr_monitor = LearningRateMonitor(logging_interval="epoch")

trainer = Trainer = L.Trainer(max_epochs=200, logger=logger, callbacks=[lr_monitor])
trainer.fit(model, train_dataloaders=loaders["train"], val_dataloaders=loaders["val"])

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/ms/anaconda3/envs/neuroimagen/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:639: Checkpoint directory /Users/ms/cs/ML/NeuroImagen/lightning_logs/ContrastiveLossFeatureLearning/Adam_0.0001_LambdaLR_margin_1.5/weight-decay_0_lambda-factor_0.95/checkpoints exists and is not empty.

  | Name    | Type                          | Params
----------------------------------------------------------
0 | loss_fn | TripletMarginWithDistanceLoss | 0     
1 | lstm    | LSTM                          | 132 K 
2 | output  | Sequential                    | 16.5 K
----------------------------------------------------------
148 K     Trainable params
0         Non-trainable params
148 K     Total params
0.594     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/ms/anaconda3/envs/neuroimagen/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/Users/ms/anaconda3/envs/neuroimagen/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


/Users/ms/anaconda3/envs/neuroimagen/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 3:   5%|▌         | 26/497 [00:02<00:39, 12.05it/s, v_num=0.95, train_loss_step=1.350, lr_step=8.57e-5, val_loss=1.500, train_loss_epoch=1.500, lr_epoch=9.02e-5] 

/Users/ms/anaconda3/envs/neuroimagen/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


# Classifier Model


In [101]:
path = "lightning_logs/ContrastiveLossFeatureLearning/Adam_1e-4_LambdaLR_0.95/version_8/checkpoints/epoch=15-step=7952.ckpt"

CKPT_PATH = os.path.join(root_path, path)

In [116]:
from torchmetrics.functional import accuracy


class EEG_Classifier(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()

        # self.feature_extractor = FeatureExtractor_ContrastiveLearning_NN()
        self.feature_extractor = (
            FeatureExtractor_ContrastiveLearning_NN.load_from_checkpoint(CKPT_PATH)
        )
        self.feature_extractor.requires_grad_(False)
        self.classifier = nn.Linear(128, 40)

        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input):
        # for p in self.feature_extractor.parameters():
        #     p.requires_grad = False
        # with torch.no_grad():
        #     input = self.feature_extractor(input)
        input = self.feature_extractor(input)
        res = self.classifier(input)
        return res

    def training_step(self, batch, batch_idx):
        eegs, labels = batch
        eegs = eegs.to(device)
        labels = labels.to(device)

        out = self(eegs)

        loss = self.loss_fn(out, labels)
        self.log_dict(
            {"train_loss": loss, "lr": self.scheduler.get_last_lr()[0]},
            on_epoch=True,
            prog_bar=True,
        )
        return loss

    def validation_step(self, batch, batch_idx):
        self._shared_eval(batch, prefix="val")

    def test_step(self, batch, batch_idx):
        self._shared_eval(batch, prefix="test")

    def _shared_eval(self, batch, prefix="val"):
        eegs, labels = batch
        out = self(eegs)
        loss = self.loss_fn(out, labels)
        preds = torch.argmax(out, dim=1)
        acc = accuracy(preds, labels, "multiclass", num_classes=preds.shape[0])
        self.log_dict({f"{prefix}_loss": loss, f"{prefix}_acc": acc}, prog_bar=True)

    def predict_step(self, batch):
        return self(batch)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.LambdaLR(
            optimizer, (lambda epoch: 0.90**epoch)
        )
        self.scheduler = scheduler
        return [optimizer], [scheduler]

# Train Classifier


In [117]:
model = EEG_Classifier()
model.to(device)

logger = TensorBoardLogger(
    save_dir="/Users/ms/cs/ML/NeuroImagen/lightning_logs/ContrastiveLossClassification",
    name="Adam_1e-3_LambdaLR_0.90",
    version=None,
)
lr_monitor = LearningRateMonitor(logging_interval="epoch")

trainer = L.Trainer(max_epochs=200, logger=logger, callbacks=[lr_monitor])
trainer.fit(model, train_dataloaders=loaders["train"], val_dataloaders=loaders["val"])

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /Users/ms/cs/ML/NeuroImagen/lightning_logs/ContrastiveLossClassification/Adam_1e-3_LambdaLR_0.90

  | Name              | Type                                    | Params
------------------------------------------------------------------------------
0 | feature_extractor | FeatureExtractor_ContrastiveLearning_NN | 148 K 
1 | classifier        | Linear                                  | 5.2 K 
2 | loss_fn           | CrossEntropyLoss                        | 0     
------------------------------------------------------------------------------
5.2 K     Trainable params
148 K     Non-trainable params
153 K     Total params
0.615     Total estimated model params size (MB)


Epoch 0:  42%|████▏     | 207/497 [00:06<00:09, 29.84it/s, v_num=0, train_loss_step=3.720, lr_step=0.001]

# Test Classifier


In [118]:
trainer.test(model, dataloaders=loaders["test"])

/Users/ms/anaconda3/envs/neuroimagen/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/Users/ms/anaconda3/envs/neuroimagen/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 124/124 [00:04<00:00, 25.95it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │   0.019657257944345474    │
│         test_loss         │    3.7022695541381836     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 3.7022695541381836, 'test_acc': 0.019657257944345474}]